# Model Evauation
Classification models need the following metrics

Cross Validation
- Accuracy
- error

ROC Curve
Steps
- load data
- featurize
- choose model
- pass data and HP's
- run CV
- ROC possible?
- tabulate results
- report

In [1]:
#### Standard Libraries ####
import os
from pprint import pprint
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial
from itertools import product
import timeit
import uuid

#### third-party Libraries ####
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier as SRC
from lolopy.learners import RandomForestClassifier as LRC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#### Local Libraries ####
from utils import (Result, run_k_folds, 
                   report_column_labels,
                   compile_data)
from data_manager import DataManager
from featurizer import Featurizer

In [2]:
# configuration
np.random.seed(8)
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('results','testing_dataramp_groupkfolds.csv')
mp_api_key = '7n6DwPUQ5cf8ZTWO'
oversample = False
data_ramp = True
feature_set = ['standard']

## Load data
This object should handle data load and sampling.

Should be able to do the following:
1. load and sample 
2. Run a data ramp

In [3]:
# Load Data
dm = DataManager(load_path, save_path)
dm.load()

'Loaded 2572 records.'


In [4]:
# Sample data
#dm.sample_data(100)

In [5]:
# Format and careate composition objects
#dm.compute_formula()
dm.to_binary_classes()
dm.get_pymatgen_composition()
dm.remove_noble_gasses()
dm.remove_features()

In [6]:
f = Featurizer(feature_set)

In [7]:
dm.featurized_data = f.featurize(dm.data)

In [8]:
dm.groups = dm.data['group']

In [9]:
# Set training labels
dm.outputs = dm.data['stable']

In [10]:
k_folds = partial(run_k_folds, inputs=dm.featurized_data, outputs=dm.outputs, groups=dm.groups, sampling=oversample, ramp=data_ramp)

In [11]:
# set models
#models = [GaussianNB(), SVC(), SRC(), LogisticRegression(), DummyClassifier(strategy="most_frequent")]
models = [GaussianNB()]

In [12]:
%%capture
pool = mp.Pool(processes=mp.cpu_count())

In [13]:
%%capture
start_time = timeit.default_timer()
results = pool.map(k_folds, models)    
elapsed = timeit.default_timer() - start_time

In [19]:
compiled

[[['GaussianNB',
   4114,
   0.8949838187702266,
   0.023034323437469188,
   0.28561543104600584,
   0.046669771113109444,
   0.4720028629220646,
   0.07958089964327215,
   0.2097122589191684,
   0.047525145286345966],
  ['GaussianNB',
   8228,
   0.8193945064788783,
   0.012367689559519897,
   0.4250158793195265,
   0.04613940084188856,
   0.5925494183391309,
   0.05745171289638946,
   0.3321737610552503,
   0.04183920729845365],
  ['GaussianNB',
   12342,
   0.7579800392391026,
   0.01913915467129765,
   0.42581492485885386,
   0.022788491858477627,
   0.6737048508002769,
   0.04166896065021293,
   0.3117912793898035,
   0.020684594012844858],
  ['GaussianNB',
   16456,
   0.7343199872953499,
   0.0226552383042466,
   0.39788455599465705,
   0.02253589646389047,
   0.6603351885677993,
   0.04453057160915248,
   0.28516710384867094,
   0.018938489153027722],
  ['GaussianNB',
   20570,
   0.7482741857073407,
   0.014338883974821392,
   0.3916131080985014,
   0.02173025814332172,
   0.6

In [20]:
compiled = []
for result in results:
    compiled.extend(compile_data(result))
res_df = pd.DataFrame(compiled, columns=report_column_labels)
res_df.to_csv(save_path)
res_df

,type,data_size,accuracy,accuracy_std,f1,f1_std,recall,recall_std,precision,precision_std
0,GaussianNB,4114,0.894984,0.023034,0.285615,0.046670,0.472003,0.079581,0.209712,0.047525
1,GaussianNB,8228,0.819395,0.012368,0.425016,0.046139,0.592549,0.057452,0.332174,0.041839
2,GaussianNB,12342,0.757980,0.019139,0.425815,0.022788,0.673705,0.041669,0.311791,0.020685
3,GaussianNB,16456,0.734320,0.022655,0.397885,0.022536,0.660335,0.044531,0.285167,0.018938
4,GaussianNB,20570,0.748274,0.014339,0.391613,0.021730,0.648737,0.049388,0.280810,0.016082


# Single model test

In [ ]:
from sklearn.utils import resample
from collections import Counter
def local_oversample(train, outputs):
    majority_class, minority_class = Counter(outputs[train]).most_common()
    minority = numpy.where(outputs[train] == minority_class[0])[0]
    majority = numpy.where(outputs[train] == majority_class[0])[0]
    minority_upsampled = resample(minority,
                              replace=True,
                              n_samples=len(majority),
                              random_state=8)                          
    return (numpy.append(minority, minority_upsampled), minority, majority, minority_upsampled)

In [ ]:
model = SVC()
ovs = True

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import KFold, GroupKFold
inputs = X
outputs = Y
groups = dm.data['group']
gkf = GroupKFold(n_splits=10)
splits = list(gkf.split(inputs, outputs, groups=groups))

In [ ]:
%%capture
train, test = splits[0]
if ovs:
    train, minority, majority, minority_upsampled = local_oversample(train, outputs)
train_df = pandas.DataFrame(dm.data.ix[train, ['formula','group','stable']])
test_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])

model.fit(inputs[train], outputs[train])
prediction = model.predict(inputs[test])
pred_df = pandas.DataFrame(dm.data.ix[test, ['formula','group','stable']])
pred_df['prediction'] = prediction
acc = accuracy_score(outputs[test], prediction)
acc_df = pandas.DataFrame([acc], columns=['accuracy'])


In [ ]:
f1s = [precision_recall_fscore_support(
    outputs[test], prediction, labels=[0,1], average='binary')]
f1_df = pandas.DataFrame(
        f1s, columns=['precision', 'recall', 'f1', 'support'])

In [ ]:
ov_df = pd.DataFrame([train_df['group'].isin(test_df['group']).any()], columns=['overlap'])

try:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0','pred_1'])
except:
    lab_df = pandas.DataFrame([pred_df['prediction'].value_counts().values], columns=['pred_0'])
if ovs:
    sample_df = pandas.DataFrame([train_df['stable'].value_counts().values], columns=['smpl_0','smpl_1'])
else:
    sample_df = pandas.DataFrame([], columns=['sample'])
ac_lab_df = pandas.DataFrame([test_df['stable'].value_counts().values], columns=['acc_0','acc_1'])

f1_df.join(acc_df).join(ov_df).join(lab_df).join(ac_lab_df).join(sample_df)

In [ ]:
pprint(f"Minority: {len(minority)}")
pprint(f"Majority: {len(majority)}")
pprint(f"Minority Upsampled: {len(minority_upsampled)}")